In [1]:
# import relevant packages

import pandas as pd
import os
import re

from html import unescape
from sklearn.model_selection import train_test_split

pd.set_option('display.max_colwidth', None)

## Load raw datasets

In [19]:
df_dict = dict()

PATH = "../0_data/main/0_raw"

for file in sorted(os.listdir(PATH)):
    if "ipynb" not in file:
        print(re.sub('\.csv$', '', file))
        df_dict[re.sub('\.csv$', '', file)] = pd.read_csv(f"{PATH}/{file}")

bas19_es
dyn21_en
for19_pt
fou18_en
has19_hi
has20_hi
has21_hi
ken20_en
ous19_ar
ous19_fr
san20_it


## Reformat columns
Need separate logic for different datasets. 1 is for hateful, 0 for non-hateful.

In [20]:
# Dynabench 2021 / English
df_dict["dyn21_en"].label.replace({"hate":1, "nothate":0}, inplace=True)

# Founta 2018 / English
df_dict["fou18_en"].label.replace({'hateful': 1, "abusive": 0, "normal": 0, "spam": 0}, inplace = True)

# Kennedy 2020 / English
df_dict["ken20_en"].rename(columns={"label_hate_maj": "label"}, inplace=True)

# Fortuna 2019 / Portuguese
df_dict["for19_pt"].rename(columns={"hatespeech_comb": "label"}, inplace=True)

# Basile 2019 / Spanish
df_dict["bas19_es"].rename(columns={"HS": "label"}, inplace=True)

# Sanguinetti 2020 / Italian
df_dict["san20_it"].rename(columns={"hs": "label"}, inplace=True)

# Ousidhoum 2019 / Arabic & French
for d in ["ous19_ar", "ous19_fr"]:
    df_dict[d]["label"] = df_dict[d].sentiment.apply(lambda x: 1 if "hateful" in x else 0)
    # text was already cleaned in a way that conflicts with our later cleaning, so we align it here
    df_dict[d]["text"] = df_dict[d].tweet.apply(lambda x: x.replace("@url", "http"))
    
# HASOC 19, 20 and 21 / Hindi
for d in ["has19_hi", "has20_hi", "has21_hi"]:
    df_dict[d]["label"] = df_dict[d].task_2.apply(lambda x: 1 if x=="HATE" else 0)
    
# drop redundant columns
for dataset in df_dict:
    if "split" in df_dict[dataset].columns:
        df_dict[dataset] = df_dict[dataset][["text", "label", "split"]]
    else:
        df_dict[dataset] = df_dict[dataset][["text", "label"]]

## Clean text

In [21]:
def clean_text(text):
    text = unescape(text)
    text = re.sub(r"@[A-Za-z0-9_-]+",'@user',text) # format expected by XLM-T
    text = re.sub(r"http\S+",'http',text) # format expected by XLM-T
    text = re.sub(r"\n",' ',text)
    text = re.sub(r"\t",' ',text)
    text = text.replace("[URL]", "http") # format expected by XLM-T
    text = text.strip()
    return text

for dataset in df_dict:
    df_dict[dataset]["text"] = df_dict[dataset].text.apply(lambda x: clean_text(x))

## Boost proportion of hate in English datasets to match Dynabench
We are doing a first phase of fine-tuning on up to 20k entries in Dynabench. Dynabench has ca. 53% hate. The other English datasets have a lower proportion of hate so we drop non-hate to make the proportion more comparable at 20k entries.

In [22]:
# boost Kennedy 2020 / English to have 50% hate (up from ca. 30%)
df_dict["ken20_en"] = pd.concat([df_dict["ken20_en"][df_dict["ken20_en"].label==1], df_dict["ken20_en"][df_dict["ken20_en"].label==0].sample(11596, random_state=123)]).sample(frac=1, random_state=123)

# boost Founta 2018 / English to have 22% hate, which is max possible (up from ca. 5%)
df_dict["fou18_en"] = pd.concat([df_dict["fou18_en"][df_dict["fou18_en"].label==1], df_dict["fou18_en"][df_dict["fou18_en"].label==0].sample(17600, random_state=123)]).sample(frac=1, random_state=123)


## Show descriptive stats

In [23]:
def descriptive_stats(df):
    n_total = df.shape[0]
    n_hate = df.label.sum()
    print("{} entries, of which {} ({:.2%}) are hateful.".format(n_total, n_hate, n_hate/n_total))
    return df.label.sum()/len(df), len(df)

for dataset in df_dict:
    print(dataset.upper())
    descriptive_stats(df_dict[dataset])
    display(df_dict[dataset].sample(5))
    print()

BAS19_ES
6600 entries, of which 2739 (41.50%) are hateful.


,text,label
1490,"Así como a muchos les encabrona que les digan negros a mi tambien me encabrona y doble que me digan güera, para mí no es un alago ni mucho menos superioridad",1
4134,"📣🗡️#Barcelona, ciudad de #okupas, inmigrantes ilegales #manteros, #turismofobia y #narcopisos en el Raval. Magrebíes atacan a un filipino en ese barrio. @user desaparecida y sin dar la cara. http",1
2243,Lo único que debe votar una mujer son sus tetas cuando hacen deporte. #GravedadAnteTodo,1
5057,#NOME?,0
240,"@user @user @user @user Pues eso, cállate la puta boca.",0



DYN21_EN
41144 entries, of which 22175 (53.90%) are hateful.


,text,label,split
18489,Pandering to the mentally ill never lasts. A society that persists in celebrating illness is doomed.,1,train
19181,Pissing me the fuck OFF! Trampy girls,0,train
10858,"I will be honest, all bl@user deserve to be saved from this racist government and I wanna do it",0,test
26502,"i want to dedicate myself to the TOTAL and complete eradication of climate change on this planet, I would dedicate an entire life to this worthy pursuit using my own hands, for these gas guzzlers among us, must be eventually dealt with en masse! Otherwise all of HUMANITY must eventually suffer under the cruetly of their fumes.",0,train
13772,"One thought: Is it possible that since the government-sponsored imprisonement of black people based upon fictitious statistics is ""unleashing & emboldening"" the most violent factions fo BLM? I'm not justifying the violence but providing a a broader context for their demands and the hsitorcal unjustices they've suffered -just a thought.",0,train



FOR19_PT
5670 entries, of which 1788 (31.53%) are hateful.


...

,text,label
390,@user @user الا خنازير قطر يقولون ايران دوله شريفه و خليفة الشواذ المتحالف معهم,1
644,@user @user هههههههههههههههههههههههههههههههههههههههههههههههه صاروخ بول البعير إحنا اللي دقينا خشومكم… http,1
2313,@user @user عالم شيعي يوصي بشرب بول البعير http,1
2101,وبصفتنا مغاربة لا نقبل دعم من خاننا في يوم الايام وكان يشتمنا يا شارب بول البعير http,1
2303,@user @user هذا القرضاوي خنزير ونصاب وقد وجد ضالته في حكام قطر الأغبياء وسيطر عليهم وأخذ من أموال الش… http,1



OUS19_FR
4014 entries, of which 399 (9.94%) are hateful.


,text,label
2248,@user @user @user @user ils mangent pas de porc parce que quand il les tuaie... http,0
2246,@user @user et ouais des mongol à marseille,0
3166,peut-être que ceux qui émettent les oukases non violents sont relativement protégés de la violence. mais il semble… http,0
3706,@user @user « @user islamo - gauchiste ? »,0
3545,@user @user @user @user encore un autre mongol qui veut se donner bonne conscience a… http,0



SAN20_IT
8100 entries, of which 3388 (41.83%) are hateful.


,text,label,split
118,"Roma, gli spezzano il dito per rubargli fede: caccia a due rom http http",0,train
4191,"@user La bella domanda è: se sono profughi, come fanno a sapere dove rivolgersi per denunciare una persona, da parte un ministro del governo italiano???? Chi li sta guidando???? Ci scommetterei una mano che proviene tutto dal pdisastro!!!!!",1,train
1300,Inchiesta a torino - Trovato in Croazia il tesoro della “regina” rom | Liguria | Genova | Il Secolo XIX http,0,train
6917,"Ma @user dimmi ,vorresti far sbarcare questi finti rifugiati politici e finti profughi? Se lo farai ,prometto che non avrai un solo voto da noi !!!! Rispetta le idee è i principi del CAPITANO SALVINI. Non devi cedere ,tutti abbiamo un cuore ,ma no x questi accattoni. http",1,test
5604,@user @user @user Accidenti😲 A pensare che prima la Calabria era un'oasi di pace Ad esempio non c'erano mai stati sequestri di persona Solo con i migranti ospitati a #Riace sono iniziati questi episodi N.B. rispetto i calabresi la cui maggioranza è formata da brave persone,0,train


## Create and export splits

In [24]:
# set aside 2k from each dataset for testing and 500 for dev
# except for Ousidhoum in French and Arabic, where train set would otherwise be too small
# and for HASOC 20 and 21 in Hindi, where test splits are given

TEST_SIZE = 2000
DEV_SIZE = 500

for dataset in df_dict:
    if "ous19_fr" in dataset:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = 2000, random_state=123)
        devset, testset = train_test_split(devtest, test_size = 1500, random_state=123)
        devset.to_csv(f"../0_data/main/1_clean/{dataset}/dev_500.csv", index=False)
        testset.to_csv(f"../0_data/main/1_clean/{dataset}/test_1500.csv", index=False)
    elif "ous19_ar" in dataset:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = 1300, random_state=123)
        devset, testset = train_test_split(devtest, test_size = 1000, random_state=123)
        devset.to_csv(f"../0_data/main/1_clean/{dataset}/dev_300.csv", index=False)
        testset.to_csv(f"../0_data/main/1_clean/{dataset}/test_1000.csv", index=False)
    elif "has19_hi" in dataset or "has20_hi" in dataset: # use provided test sets
        df_dict[dataset][df_dict[dataset]["split"]=="test"].to_csv(f"../0_data/main/1_clean/{dataset}/test_{len(df_dict[dataset][df_dict[dataset]['split']=='test'])}.csv", index=False)
        df_dict[dataset], devset = train_test_split(df_dict[dataset][df_dict[dataset]["split"]=="train"], test_size = 500, random_state=123)
        devset.to_csv(f"../0_data/main/1_clean/{dataset}/dev_500.csv", index=False)
    else:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = TEST_SIZE+DEV_SIZE, random_state=123)
        devset, testset = train_test_split(devtest, test_size = TEST_SIZE, random_state=123)
        devset.to_csv(f"../0_data/main/1_clean/{dataset}/dev_{DEV_SIZE}.csv", index=False)
        testset.to_csv(f"../0_data/main/1_clean/{dataset}/test_{TEST_SIZE}.csv", index=False)
        
# export all data that is not test or dev, so we can use it for full sample training
for dataset in df_dict:
    df_dict[dataset].to_csv(f"../0_data/main/1_clean/{dataset}/train_{len(df_dict[dataset])}.csv", index=False)

In [25]:
# create differently-sized train portions from rest of data

SEEDS = 10 # for repeated experiments with different random data selection
N_RANGE = [10, 20, 30, 40, 50, 100, 200, 300, 400, 500, 1000, 2000, 3000, 4000, 5000, 10000, 20000]

for dataset in df_dict:
    print(dataset.upper())
    for n in N_RANGE:
        
        # save all splits for English test sets
        if n<len(df_dict[dataset]) and ("dyn21" in dataset or "ken20" in dataset or "fou18" in dataset): 
            print(f"  saving n = {n} training set")
            for random_state in range(1, SEEDS+1):
                df_dict[dataset].sample(n, random_state = random_state).to_csv(f"../0_data/main/1_clean/{dataset}/train/train_{n}_rs{random_state}.csv",index=False)
        
        # save splits up to 2k for other datasets
        elif n<len(df_dict[dataset]) and n<=2000: 
            print(f"  saving n = {n} training set")
            for random_state in range(1, SEEDS+1):  
                df_dict[dataset].sample(n, random_state = random_state).to_csv(f"../0_data/main/1_clean/{dataset}/train/train_{n}_rs{random_state}.csv",index=False)
    
    print()

BAS19_ES
  saving n = 10 training set
  saving n = 20 training set
  saving n = 30 training set
  saving n = 40 training set
  saving n = 50 training set
  saving n = 100 training set
  saving n = 200 training set
  saving n = 300 training set
  saving n = 400 training set
  saving n = 500 training set
  saving n = 1000 training set
  saving n = 2000 training set

DYN21_EN
  saving n = 10 training set
  saving n = 20 training set
  saving n = 30 training set
  saving n = 40 training set
  saving n = 50 training set
  saving n = 100 training set
  saving n = 200 training set
  saving n = 300 training set
  saving n = 400 training set
  saving n = 500 training set
  saving n = 1000 training set
  saving n = 2000 training set
  saving n = 3000 training set
  saving n = 4000 training set
  saving n = 5000 training set
  saving n = 10000 training set
  saving n = 20000 training set

FOR19_PT
  saving n = 10 training set
  saving n = 20 training set
  saving n = 30 training set
  saving n = 